# Clustering neighborhoods in Toronto

1. get neighborhoods in Toronto
2. explore venues among neighborhoods
3. clustering neighborhoods based on categories distribution of their venues

## Part I. scrape pastal code: 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [22]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"619aa236-74a4-4d93-b682-88d9dbe449b8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [23]:
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    if cols:
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols]) # Get rid of empty values

In [24]:
# convert data into a dataframe
import pandas as pd
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [25]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# df['Borough'].unique() #display unique Borough to make sure no other variation of "Not assigned"
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
df.shape

(103, 3)

## Part II. latitude and the longitude coordinates of each neighborhood

In [84]:
# !conda install -c conda-forge geopy --yes 
# !pip install geocoder

In [1]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# postal_code = df['Borough'][3]
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [31]:
import numpy as np
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# geolocator = Nominatim(user_agent="ca_explorer")
# # address = 'New York City, NY'

# latitude = np.empty(df.shape[0])
# longitude = np.empty(df.shape[0])
# latitude[:] = np.nan
# longitude[:] = np.nan

# for ind in range(df.shape[0]):
#     address = '{}, {}, Toronto, Ontario'.format(df['Neighborhood'][ind].split(',')[0],df['Borough'][ind])
#     postal_code = df['PostalCode'][ind]

#     location = geolocator.geocode(address)
#     if location:
#         latitude[ind] = location.latitude
#         longitude[ind] = location.longitude
#     print('The geograpical coordinate of {} ({}) are {}, {}.'.format(address, postal_code, latitude[ind], longitude[ind]))
    
# np.sum(np.isnan(longitude))    

In [7]:
# add new columns to df
# df['Latitude'] = latitude
# df['Longitude'] = longitude
# df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663217,-79.386290


In [26]:
# alternatively, load coordinates data directly...
postal_data = pd.read_csv('http://cocl.us/Geospatial_data') 
postal_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
postal_data.columns = ['PostalCode','Latitude','Longitude']

In [20]:
df_to = pd.merge(df, postal_data, how='left', on='PostalCode')
df_to.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [28]:
# !conda install -c conda-forge folium=0.5.0 --yes
import folium

address = 'Toronto, Ontario'
location = geolocator.geocode(address)
latitude_to = location.latitude
longitude_to = location.longitude

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_to, longitude_to], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_to['Latitude'], df_to['Longitude'], df_to['Borough'], df_to['Neighborhood']):
    if ~np.isnan(lat):
        label = '{}; {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
    else:
        print('Missing: {}; {}.'.format(neighborhood,borough))
    
map_toronto

## Part III. explore the neighborhood and to cluster

In [37]:
# function to find out venues near each [latitudes, longitudes]
CLIENT_ID = 'KDBNNYY3GCQDW5MEI2U0KV1TEPMFC0QIRD2DTEVZHLUB0DAV'
CLIENT_SECRET = 'DYZCEQBM2GJLIXZDLR11LEGBWQRSXRGLB02Y35HWDFCEMKCU' # your Foursquare Secret
VERSION = '20180605' 
LIMIT = 50

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
toronto_venues = getNearbyVenues(names=df_to['Neighborhood'][:20],
                                   latitudes=df_to['Latitude'][:20],
                                   longitudes=df_to['Longitude'][:20]
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches


In [54]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...
270,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
271,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
272,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
273,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [55]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Don Mills,26,26,26,26,26,26
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",9,9,9,9,9,9
"Garden District, Ryerson",50,50,50,50,50,50
Glencairn,5,5,5,5,5,5
"Guildwood, Morningside, West Hill",7,7,7,7,7,7
Humewood-Cedarvale,4,4,4,4,4,4
"Lawrence Manor, Lawrence Heights",15,15,15,15,15,15
"Malvern, Rouge",1,1,1,1,1,1
"Parkview Hill, Woodbine Gardens",10,10,10,10,10,10


In [87]:
# for each neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Sushi Restaurant,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,Don Mills,0.000000,0.000000,0.00,0.000000,0.038462,0.000000,0.076923,0.038462,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
2,"Garden District, Ryerson",0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.02,0.04,0.02,0.040000,0.00,0.00,0.000000,0.000000
3,Glencairn,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
4,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
5,Humewood-Cedarvale,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.25,0.00,0.000000,0.000000
6,"Lawrence Manor, Lawrence Heights",0.000000,0.133333,0.00,0.000000,0.000000,0.066667,0.000000,0.066667,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.066667,0.066667
7,"Malvern, Rouge",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
8,"Parkview Hill, Woodbine Gardens",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000
9,Parkwoods,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000


In [58]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Don Mills----
                 venue  freq
0           Beer Store  0.08
1          Coffee Shop  0.08
2  Japanese Restaurant  0.08
3     Asian Restaurant  0.08
4                  Gym  0.08


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
               venue  freq
0  Convenience Store  0.11
1        Coffee Shop  0.11
2        Pizza Place  0.11
3           Pharmacy  0.11
4          Pet Store  0.11


----Garden District, Ryerson----
                 venue  freq
0                 Café  0.06
1  Japanese Restaurant  0.04
2       Clothing Store  0.04
3       Cosmetics Shop  0.04
4              Theater  0.04


----Glencairn----
                 venue  freq
0   Italian Restaurant   0.2
1                  Pub   0.2
2     Asian Restaurant   0.2
3  Japanese Restaurant   0.2
4               Bakery   0.2


----Guildwood, Morningside, West Hill----
               venue  freq
0     Medical Center  0.14
1       Intersection  0.14
2  Electronics Store  0.14
3               Ban

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Don Mills,Japanese Restaurant,Coffee Shop,Restaurant,Gym,Asian Restaurant
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pizza Place,Beer Store,Coffee Shop,Convenience Store,Café
2,"Garden District, Ryerson",Café,Cosmetics Shop,Ramen Restaurant,Japanese Restaurant,Coffee Shop
3,Glencairn,Japanese Restaurant,Bakery,Italian Restaurant,Pub,Asian Restaurant
4,"Guildwood, Morningside, West Hill",Rental Car Location,Mexican Restaurant,Medical Center,Electronics Store,Intersection


In [63]:
# cluster
from sklearn.cluster import KMeans


# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_to

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Construction & Landscaping,Park,Food & Drink Shop,Women's Store,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Pizza Place
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Pub,Park,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Furniture / Home Store,Women's Store,Athletics & Sports
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Beer Bar


In [85]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[latitude_to, longitude_to], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='#000000' if np.isnan(cluster) else rainbow[int(cluster)-1],
        fill=True,
        fill_color='#000000' if np.isnan(cluster) else rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [86]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Construction & Landscaping,Park,Food & Drink Shop,Women's Store,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Pizza Place
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Pub,Park,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Furniture / Home Store,Women's Store,Athletics & Sports
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Beer Bar
...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,NaN,NaN,NaN,NaN,NaN,NaN
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,NaN,NaN,NaN,NaN,NaN,NaN
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,NaN,NaN,NaN,NaN,NaN,NaN
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,NaN,NaN,NaN,NaN,NaN,NaN
